# LlamaIndex, Vector Store Quickstart

Create a Vector Store with LamaIndex and CassIO, and build a powerful search engine and text generator, backed by Astra DB / Apache Cassandra®.

## Colab-specific setup

Make sure you have a Database and get ready to upload the Secure Connect Bundle and supply the Token string
(see [Pre-requisites](https://cassio.org/start_here/#vector-database) on cassio.org for details. Remember you need a **custom Token** with role [Database Administrator](https://awesome-astra.github.io/docs/pages/astra/create-token/)).

Likewise, ensure you have the necessary secret for the LLM provider of your choice: you'll be asked to input it shortly
(see [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for details).

_Note: this notebook is part of the CassIO documentation. Visit [this page on cassIO.org](https://cassio.org/frameworks/llamaindex/vector-quickstart/)._


In [ ]:
# install required dependencies
! pip install \
    "cassio>=0.1.0" \
    "google-cloud-aiplatform>=1.25.0" \
    "jupyter>=1.0.0" \
    "llama-index==0.8.8" \
    "nltk==3.8.1" \
    "pypdf==3.15.2" \
    "python-dotenv==1.0.0"

You will likely be asked to "Restart the Runtime" at this time, as some dependencies
have been upgraded. **Please do restart the runtime now** for a smoother execution from this point onward.

In [ ]:
# Input your database keyspace name:
ASTRA_DB_KEYSPACE = input('Your Astra DB Keyspace name (e.g. cassio_tutorials): ')

In [ ]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
from getpass import getpass
ASTRA_DB_TOKEN_BASED_PASSWORD = getpass('Your Astra DB Token ("AstraCS:..."): ')

### Astra DB Secure Connect Bundle

Please upload the Secure Connect Bundle zipfile to connect to your Astra DB instance.

The Secure Connect Bundle is needed to establish a secure connection to the database.
Click [here](https://awesome-astra.github.io/docs/pages/astra/download-scb/#c-procedure) for instructions on how to download it from Astra DB.

In [ ]:
# Upload your Secure Connect Bundle zipfile:
import os
from google.colab import files


print('Please upload your Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
    )

In [ ]:
# colab-specific override of helper functions
from cassandra.cluster import (
    Cluster,
)
from cassandra.auth import PlainTextAuthProvider

# The "username" is the literal string 'token' for this connection mode:
ASTRA_DB_TOKEN_BASED_USERNAME = 'token'


def getCQLSession(mode='astra_db'):
    if mode == 'astra_db':
        cluster = Cluster(
            cloud={
                "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
            },
            auth_provider=PlainTextAuthProvider(
                ASTRA_DB_TOKEN_BASED_USERNAME,
                ASTRA_DB_TOKEN_BASED_PASSWORD,
            ),
        )
        astraSession = cluster.connect()
        return astraSession
    else:
        raise ValueError('Unsupported CQL Session mode')

def getCQLKeyspace(mode='astra_db'):
    if mode == 'astra_db':
        return ASTRA_DB_KEYSPACE
    else:
        raise ValueError('Unsupported CQL Session mode')

### LLM Provider

In the cell below you can choose between **GCP Vertex AI** or **OpenAI** for your LLM services.
(See [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for more details).

Make sure you set the `llmProvider` variable and supply the corresponding access secrets in the following cell.

In [ ]:
# Set your secret(s) for LLM access:
llmProvider = 'OpenAI'  # 'GCP_VertexAI', 'Azure_OpenAI'


In [ ]:
from getpass import getpass
if llmProvider == 'OpenAI':
    apiSecret = getpass(f'Your secret for LLM provider "{llmProvider}": ')
    os.environ['OPENAI_API_KEY'] = apiSecret
elif llmProvider == 'GCP_VertexAI':
    # we need a json file
    print(f'Please upload your Service Account JSON for the LLM provider "{llmProvider}":')
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        vertexAIJsonFileTitle = list(uploaded.keys())[0]
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.getcwd(), vertexAIJsonFileTitle)
    else:
        raise ValueError(
            'No file uploaded. Please re-run the cell.'
        )
elif llmProvider == 'Azure_OpenAI':
    # a few parameters must be input
    apiSecret = input(f'Your API Key for LLM provider "{llmProvider}": ')
    os.environ['AZURE_OPENAI_API_KEY'] = apiSecret
    apiBase = input('The "Base URL" for your models (e.g. "https://YOUR-RESOURCE-NAME.openai.azure.com"): ')
    os.environ['AZURE_OPENAI_API_BASE'] = apiBase
    apiLLMDepl = input('The name of your LLM Deployment: ')
    os.environ['AZURE_OPENAI_LLM_DEPLOYMENT'] = apiLLMDepl
    apiLLMModel = input('The name of your LLM Model (e.g. "gpt-4"): ')
    os.environ['AZURE_OPENAI_LLM_MODEL'] = apiLLMModel
    apiEmbDepl = input('The name for your Embeddings Deployment: ')
    os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT'] = apiEmbDepl
    apiEmbModel = input('The name of your Embedding Model (e.g. "text-embedding-ada-002"): ')
    os.environ['AZURE_OPENAI_EMBEDDINGS_MODEL'] = apiEmbModel

    # The following is probably not going to change for some time...
    os.environ['AZURE_OPENAI_API_VERSION'] = '2023-03-15-preview'
else:
    raise ValueError('Unknown/unsupported LLM Provider')

In [ ]:
# retrieve a couple of PDF files to index in the vector store
! mkdir pdfs
! curl https://raw.githubusercontent.com/CassioML/cassio-website/main/docs/frameworks/llamaindex/pdfs/rapunzel.pdf --output pdfs/rapunzel.pdf
! curl https://raw.githubusercontent.com/CassioML/cassio-website/main/docs/frameworks/llamaindex/pdfs/snow.pdf --output pdfs/snow.pdf

### Colab preamble completed

The following cells constitute the demo notebook proper.

# LlamaIndex, Vector Store Quickstart

Create a Vector Store with LamaIndex and CassIO, and build a powerful search engine and text generator, backed by Astra DB / Apache Cassandra®.

### Table of contents:

- Setup (database, LLM+embeddings)
- Create vector store
- Insert documents in it
- Answer questions using Vector Search
- Remove documents
- Cleanup

_**NOTE:** this uses Cassandra's "Vector Similarity Search" capability.
Make sure you are connecting to a vector-enabled database for this demo._

## Setup

In [1]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext

This is the LlamaIndex class providing support for Astra DB / Cassandra:

In [2]:
from llama_index.vector_stores import CassandraVectorStore

A database connection is needed to access Cassandra. The following assumes
that a _vector-search-capable Astra DB instance_ is available. Adjust as needed.

In [3]:
# creation of the DB connection
cqlMode = 'astra_db'
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

Both an LLM and an embedding function are required.

Below is the logic to instantiate the LLM and embeddings of choice. We chose to leave it in the notebooks for clarity.

In [4]:
import os
# creation of the LLM resources


if llmProvider == 'OpenAI':
    os.environ['OPENAI_API_TYPE'] = 'open_ai'
    from llama_index.llms import OpenAI
    from llama_index.embeddings import OpenAIEmbedding
    llm = OpenAI(temperature=0)
    myEmbedding = OpenAIEmbedding()
    vector_dimension = 1536
    print("LLM+embeddings from OpenAI")
elif llmProvider == 'GCP_VertexAI':
    from llama_index import LangchainEmbedding
    # LlamaIndex lets you plug any LangChain's LLM+embeddings:
    from langchain.llms import VertexAI
    from langchain.embeddings import VertexAIEmbeddings
    llm = VertexAI()
    lcEmbedding = VertexAIEmbeddings()
    vector_dimension = len(lcEmbedding.embed_query("This is a sample sentence."))
    # ... if you take care of wrapping the LangChain embedding like this:
    myEmbedding = LangchainEmbedding(lcEmbedding)
    print("LLM+embeddings from Vertex AI")
elif llmProvider == 'Azure_OpenAI':
    os.environ['OPENAI_API_TYPE'] = 'azure'
    os.environ['OPENAI_API_VERSION'] = os.environ['AZURE_OPENAI_API_VERSION']
    os.environ['OPENAI_API_BASE'] = os.environ['AZURE_OPENAI_API_BASE']
    os.environ['OPENAI_API_KEY'] = os.environ['AZURE_OPENAI_API_KEY']
    from llama_index import LangchainEmbedding
    # LlamaIndex lets you plug any LangChain's LLM+embeddings:
    from langchain.llms import AzureOpenAI
    from langchain.embeddings import OpenAIEmbeddings
    llm = AzureOpenAI(temperature=0, model_name=os.environ['AZURE_OPENAI_LLM_MODEL'],
                      engine=os.environ['AZURE_OPENAI_LLM_DEPLOYMENT'])
    lcEmbedding = OpenAIEmbeddings(model=os.environ['AZURE_OPENAI_EMBEDDINGS_MODEL'],
                                   deployment=os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT'])
    vector_dimension = len(lcEmbedding.embed_query("This is a sample sentence."))
    # ... if you take care of wrapping the LangChain embedding like this:
    myEmbedding = LangchainEmbedding(lcEmbedding)
    print('LLM+embeddings from Azure OpenAI')
else:
    raise ValueError('Unknown LLM provider.')

print(f"Vector dimension for this embedding model: {vector_dimension}")

LLM+embeddings from OpenAI
Vector dimension for this embedding model: 1536


The following cell ensures that throughout all of LlamaIndex the LLM and embedding function instantiated above will be used:

In [5]:
from llama_index import ServiceContext
from llama_index import set_global_service_context

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=myEmbedding,
    chunk_size=256,
)
set_global_service_context(service_context)

## Create vector store

In [6]:
table_name = 'vs_ll_' + llmProvider

In the LlamaIndex abstractions, the `CassandraVectorStore` instance is best wrapped into the creation of a "storage context", which you'll momentarily use to create the index proper:

In [7]:
storage_context = StorageContext.from_defaults(
    vector_store = CassandraVectorStore(
        session=session,
        keyspace=keyspace,
        table=table_name,
        embedding_dimension=vector_dimension,
        insertion_batch_size=15,
    )
)

## Insert documents

You'll want to be able to employ filtering on metadata in your question-answering process, so here is a simple way to associate a metadata dictionary to the ingested sources.

LlamaIndex supports a _function_ that maps an input file name to a metadata dictionary, and will honour the latter when storing the source text along with the embedding vectors:

In [8]:
def my_file_metadata(file_name: str):
    if "snow" in file_name:
        return {"story": "snow_white"}
    elif "rapunzel" in file_name:
        return {"story": "rapunzel"}
    else:
        return {"story": "other"}

In the vector store, you can load, for example, the PDF files found in a given directory. Actually, you can load the documents _and_ instantiate the "index" object at the same time:

In [9]:
documents = SimpleDirectoryReader(
    'pdfs',
    file_metadata=my_file_metadata
).load_data()
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)

### Re-opening a preexisting vector store

In most realistic cases, you want to access a vector store that was created and populated previously. To do so, you would create the `CassandraVectorStore` as you saw already, and then use the `from_vector_store` static method of VectorStoreIndex, obtaining an "index" that you can use exactly like the one you just created:

In [10]:
# This is how you would get an index for a pre-existing LlamaIndex vector store:
vector_store = CassandraVectorStore(
    session=session,
    keyspace=keyspace,
    table=table_name,
    embedding_dimension=vector_dimension,
)
index_from_preexisting = VectorStoreIndex.from_vector_store(
    vector_store=vector_store
)
# You can try replacing "index_from_preexisting" in place of
# "index" everywhere in the next cells ... e.g.:
#     query_engine = index_from_preexisting.as_query_engine(...)

## Answer questions

Everything is ready to start asking questions. This cell allows to search for the answer over the whole indexed corpus:

In [11]:
question_1 = "Who is the antagonist of the young lady?"
print(f"\nQuestion on the whole store: {question_1}\n ==> ", end='')
query_engine_all = index.as_query_engine(similarity_top_k=6)
response_from_all = query_engine_all.query(question_1)
print(response_from_all.response.strip())


Question on the whole store: Who is the antagonist of the young lady?
 ==> The antagonist of the young lady is the queen.


### Metadata filtering

You can, instead, constrain the lookup with filters on the metadata.

Here is the same question, limited to one of the input documents through a filtering condition:

In [12]:
from llama_index.vector_stores.types import ExactMatchFilter, MetadataFilters

print(f"\nQuestion on the 'red_hood' document: {question_1}\n ==> ", end='')
query_engine_doc1 = index.as_query_engine(
    similarity_top_k=6,
    filters=MetadataFilters(filters=[
        ExactMatchFilter(key="story", value="snow_white"),
    ])
)
response_doc1 = query_engine_doc1.query(question_1)
print(response_doc1.response.strip())


Question on the 'red_hood' document: Who is the antagonist of the young lady?
 ==> The antagonist of the young lady is the queen.


And here the very same process, this time limiting the sources to the _other_ input PDF:

In [13]:
print(f"\nQuestion on the 'rapunzel' document: {question_1}\n ==> ", end='')
query_engine_doc2 = index.as_query_engine(
    similarity_top_k=6,
    filters=MetadataFilters(filters=[
        ExactMatchFilter(key="story", value="rapunzel"),
    ])
)
response_doc2 = query_engine_doc2.query(question_1)
print(response_doc2.response.strip())


Question on the 'rapunzel' document: Who is the antagonist of the young lady?
 ==> The antagonist of the young lady is the witch.


### MMR (Maximum-marginal-relevance) method

In many cases, using the MMR method enhances the quality of the answers.

In short, the method, when running the search on the source which underpins the answer generation, tries to pick chunks of text that, while still relevant, are as diverse from each other as possible.

In this cell, you can see it in action combined with metadata filtering:

In [14]:
print(f"\nQuestion on 'snow_white', MMR method: {question_1}\n ==> ", end='')
query_engine_doc2_mmr = index.as_query_engine(
    similarity_top_k=6,
    vector_store_query_mode="mmr",
    vector_store_kwargs={
        "mmr_prefetch_k": 20,
    },
    filters=MetadataFilters(filters=[
        ExactMatchFilter(key="story", value="snow_white"),
    ])
)
response_doc2_mmr = query_engine_doc2_mmr.query(question_1)
print(response_doc2_mmr.response.strip())


Question on 'snow_white', MMR method: Who is the antagonist of the young lady?
 ==> The antagonist of the young lady is the wicked queen.


## Remove documents

Sometimes you need to remove a document from the store. This generally entails removal of a _number_ of nodes (i.e. the individual chunks of text, each with its embedding vector, into which the input document is split at ingestion time).

This is made easy with the `delete` method of the vector store. _Just keep in mind that, when indexing PDF files, LlamaIndex will treat each *page* of the input file as a separate document, so that you will erase one page at a time._

In the following, you will:

 1. ask a question to check the "baseline" result
 2. get a few "document IDs" relevant to the question
 3. use the `delete` method to remove those IDs from the vector store
 4. ask the same question and compare the answer you get.

Ask a question to check the result:

In [15]:
q_removal_test = "Who is Mother Gothel?"
print(f"\nQuestion before removal: {q_removal_test}\n ==> ", end='')
response_before_deletion = query_engine_all.query(q_removal_test)
print(response_before_deletion.response.strip())


Question before removal: Who is Mother Gothel?
 ==> Mother Gothel is a character mentioned in the story "Rapunzel".


Use the `retrieve` query-engine primitive to get a "raw" list of best-match document:

In [16]:
from llama_index.indices.query.schema import QueryBundle

q_bundle = QueryBundle(query_str=q_removal_test)
query_engine_all_manydocs  = index.as_query_engine(similarity_top_k=4)
nodes_with_scores = query_engine_all_manydocs.retrieve(q_bundle)

print(f"Found {len(nodes_with_scores)} nodes with their score.")

Found 4 nodes with their score.


Now delete a few documents (remember that there are in general several "nodes" for each "document", as the `Counter` here may highlight):

In [17]:
from collections import Counter

nodes_per_document = Counter(nws.node.ref_doc_id for nws in nodes_with_scores)

for document_id, node_count in nodes_per_document.most_common():
    print(f"Deleting doc={document_id} (came up in {node_count} node(s))")
    vector_store.delete(document_id)
print("Done deleting.")

Deleting doc=22330ead-09cb-4266-a114-3f488eb94522 (came up in 2 node(s))
Deleting doc=f2e589f0-a5c7-40f8-9454-bd8aba052f37 (came up in 1 node(s))
Deleting doc=3da3723f-1f48-4974-8f9f-2e9112d51aa5 (came up in 1 node(s))
Done deleting.


Now repeat the question:

In [18]:
print(f"\nQuestion after removal: {q_removal_test}\n ==> ", end='')
response_after_deletion = query_engine_all.query(q_removal_test)
if response_after_deletion.response:
    print(response_after_deletion.response.strip())
else:
    print("(no answer received)")


Question after removal: Who is Mother Gothel?
 ==> I'm sorry, but based on the given context information, there is no mention of Mother Gothel.


## (optional) Cleanup

You may want to clean up your database: in that case, simply run the following cell.

_Warning: this will delete the vector store and all that you stored into it!_

In [ ]:
session.execute(f"DROP TABLE IF EXISTS {keyspace}.{table_name};")

## What now?

This demo is hosted [here](https://cassio.org/frameworks/llamaindex/vector-quickstart/) at cassio.org.

Discover the other ways you can integrate 
Cassandra/Astra DB with your ML/GenAI needs,
right **within [your favorite framework](https://cassio.org/frameworks/llamaindex/about/)**.